In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.externals import joblib


#read the csv file
df = pd.read_csv('data/stat_acc_V3_new.csv', sep=';')

#initialisation des variables X et Y (à modifier avec les jeux de données d'apprentissage)
X = df.drop(['descr_grav'], axis=1)
Y = df['descr_grav']

### Classification avec SVM

In [ ]:
def SVM_class(X,Y,old_model=None):
    #création d'un classifieur SVM
    svm = SVC()
    svm_params = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
    svm_grid = GridSearchCV(svm, svm_params, cv=5)
    svm_grid.fit(X, Y)
    best_svm = svm_grid.best_estimator_

    #enregistrement du modèle avec les meilleurs paramètres
    #comparaison avec l'ancien modèle pour ne pas le remplacer si moins performant
    if old_model == None:
        joblib.dump(best_svm, 'models/svm_model.pkl')
    else:
        if old_model.score(X,Y) < best_svm.score(X,Y):
            joblib.dump(best_svm, 'models/svm_model.pkl')

    return best_svm


### Classification avec Random Forest

In [ ]:
def RF_class(X,Y,old_model=None):
    #création d'un classifieur Random Forest
    rf = RandomForestClassifier()
    rf_params = {'n_estimators':[10, 100, 1000], 'max_depth':[None, 10, 100]}
    rf_grid = GridSearchCV(rf, rf_params, cv=5)
    rf_grid.fit(X, Y)
    best_rf = rf_grid.best_estimator_

    #enregistrement du modèle avec les meilleurs paramètres
    #comparaison avec l'ancien modèle pour ne pas le remplacer si moins performant
    if old_model == None:
        joblib.dump(best_rf, 'models/rf_model.pkl')
    else:
        if old_model.score(X,Y) < best_rf.score(X,Y):
            joblib.dump(best_rf, 'models/rf_model.pkl')


    return best_rf


### Classification avec MLP

In [ ]:
def MLP_class(X,Y,old_model=None):
    #création d'un classifieur MLP
    mlp = MLPClassifier()
    mlp_params = {'hidden_layer_sizes':[(10,), (100,), (1000,)], 'activation':['relu', 'tanh', 'logistic']}
    mlp_grid = GridSearchCV(mlp, mlp_params, cv=5)
    mlp_grid.fit(X, Y)
    best_mlp = mlp_grid.best_estimator_

    #enregistrement du modèle avec les meilleurs paramètres
    #comparaison avec l'ancien modèle pour ne pas le remplacer si moins performant
    if old_model == None:
        joblib.dump(best_mlp, 'models/mlp_model.pkl')
    else:
        if old_model.score(X,Y) < best_mlp.score(X,Y):
            joblib.dump(best_mlp, 'models/mlp_model.pkl')

    return best_mlp

### Fusion classifieurs

In [ ]:
#fusion des classifieurs

#vote majoritaire
voting_clf_maj = VotingClassifier(estimators=[('svm', best_svm), ('rf', best_rf), ('mlp', best_mlp)], voting='hard')
voting_clf_maj.fit(X, Y)

#vote pondéré
voting_clf_pond = VotingClassifier(estimators=[('svm', best_svm), ('rf', best_rf), ('mlp', best_mlp)], voting='soft')
voting_clf_pond.fit(X, Y)

#prediction sur un nouveau jeu de données (à modifier avec les jeux de données de test)
X_pred = X_test

#prediction vote majoritaire
Y_pred_maj = voting_clf_maj.predict(X_test)

#prediction vote pondéré
Y_pred_pond = voting_clf_pond.predict(X_test)